In [13]:
import tensorflow as tf
import numpy as np
from numpy import ndarray

In [14]:
generator = np.random.default_rng()
class Layer:
    def __init__(self,inputAmnt,outputAmnt):
        def sharmaAct(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)
            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] = (x[mask1] + 1.5) / np.sqrt(np.e)
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = x[~(mask1 | mask2)] + 1
            return result
        def sharmaActPrime(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)
            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] =  1 / np.sqrt(np.e) 
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = 1 
            return result
        self.f = sharmaAct
        self.fPrime = sharmaActPrime
        self.w = np.random.uniform(low=-1,high=1,size=(outputAmnt,inputAmnt))
        self.b = np.random.uniform(low=-1,high=1,size=(outputAmnt,1))
        self.linearCombo = 0
        self.input = 0
    def compute(self,input):
        self.input = input
        self.linearCombo = self.w @ input + self.b
        return self.f(self.linearCombo)
    def compPrime(self):
        return self.fPrime(self.linearCombo)
class inputLayer:
    def __init__(self,activation):
        def relu(input):
            return np.maximum(input,0)
        def reluPrime(input):
            return np.greater(input, 0).astype(np.float32)
        def sigmoid(input):
            return 1/(1+np.exp(-input))
        def sigmoidPrime(input):
            return -np.exp(-input)/(1+np.exp(input))
        def tanhPrime(input):
            np.power(1/(np.cosh(input)),2)
        def sharmaAct(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)

            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] = (x[mask1] + 1.5) / np.sqrt(np.e)
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = x[~(mask1 | mask2)] + 1
            return result
        def sharmaActPrime(input):
            x = input
            mask1 = x < -0.5
            mask2 = (-0.5 <= x) & (x <= 0.5)
            
            result = np.zeros_like(x)  # Initialize an array of zeros
            result[mask1] =  1 / np.sqrt(np.e) 
            result[mask2] = np.exp(x[mask2])
            result[~(mask1 | mask2)] = 1 
            return result
        val = {"relu":[relu,reluPrime],"sigmoid":[sigmoid,sigmoidPrime],"tanh":[np.tanh,tanhPrime],'sharma':[sharmaAct,sharmaActPrime]}
        self.f = val[activation][0]


class ANN:
    def __init__(self,input,*layers):
        self.layers = list(layers)
        self.input = input
    def predict(self,input: np.array):
        out = self.input.f(input)
        for k in self.layers:
            out = k.compute(out)
        return out/np.sum(np.abs(out))
    def gradientDescent(self,inputs,outputs,alpha):
        for (input,output) in zip(inputs,outputs):
            vectorPart = 2*(self.predict(input)-output)
            self.layers.reverse()
            matThing = np.identity(n=self.layers[0].w.shape[0])
            prevW = -1
            for l in self.layers:
                if(not isinstance(prevW,int)):
                    matThing = matThing @ (prevW * np.transpose(l.compPrime()))
                value = np.transpose(matThing) @ vectorPart
                l.b -= alpha * value
                l.w -= value * np.tile(np.transpose(l.input),(l.w.shape[0],1))*alpha
                prevW = l.w
            
            self.layers.reverse()

In [34]:
model = ANN(inputLayer('sharma'),Layer(784,15),Layer(15,10))


In [16]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], -1,1)  # input for training
x_test = x_test.reshape(x_test.shape[0], -1,1)  # input for testing
y_train = y_train.reshape(60_000,1)

In [17]:
def encode(input):
    z = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
    z[0][input] = 1
    return np.transpose(z)
output_test = np.apply_along_axis(encode,1,y_train)


In [141]:
model.gradientDescent(x_train,output_test,1/300_000)
    

In [137]:
#0.67935 (its been stable at about 67.5 percennt)
def percentAcc(model):
    global x_train,y_train
    error = 0
    for (input,output) in zip(x_train,y_train):
        if(np.argmax(model.predict(input)) != output):
            error += 1
    return error
print(1-percentAcc(model)/60_000)

0.6708833333333333


In [78]:
for l in model.layers:
        l.b /= np.average(np.abs(l.b))
        l.w /= np.average(np.abs(l.w))

In [144]:
model.layers[0].w

array([[ 9.78942123e+27,  9.78942123e+27,  9.78942123e+27, ...,
         9.78942123e+27,  9.78942123e+27,  9.78942123e+27],
       [ 3.79874992e+27,  3.79874992e+27,  3.79874992e+27, ...,
         3.79874992e+27,  3.79874992e+27,  3.79874992e+27],
       [ 4.14196321e+28,  4.14196321e+28,  4.14196321e+28, ...,
         4.14196321e+28,  4.14196321e+28,  4.14196321e+28],
       ...,
       [-6.46633770e+27, -6.46633770e+27, -6.46633770e+27, ...,
        -6.46633770e+27, -6.46633770e+27, -6.46633770e+27],
       [-6.25458063e+27, -6.25458063e+27, -6.25458063e+27, ...,
        -6.25458063e+27, -6.25458063e+27, -6.25458063e+27],
       [ 6.79042712e+28,  6.79042712e+28,  6.79042712e+28, ...,
         6.79042712e+28,  6.79042712e+28,  6.79042712e+28]])